In [ ]:
# -*- coding: utf-8 -*-
import random
import networkx as nx
import numpy as np
import json
import time

# ----------start to count time-------------------
beginTime = time.clock()

# Inverse Transform Method
# more efficient
def powerlaw_num(v, xmin=0.02):
    x = random.uniform(0, 1./(1+v)*xmin**(-1-v))
    return (xmin**(-1-v)-(1+v)*x)**(-1./(1+v))

# Generating an empty graph
def emptyGraph(N):
    G = nx.Graph()
    G.add_nodes_from(range(N))
    return G

# Generating activity driven time-varying network in one timeslot
def init_activityDriven(N=1000, m=6):
    """"
    Parameters
    ----------
    N :   nodes
    m :   rewiring links
    """
    # initiate parameters
    # N = 1000   # nodes
    # m = 6     # rewiring links
    dt = 1    # active slot
    eta = 10  # rescaling factor , ai = eta * xi

    G = nx.Graph()
    G.add_nodes_from(range(N))
    for i in range(N):
        xi = powerlaw_num(3, 0.03)  # node's activated prob belongs to F(x)=x**(-3)
        if random.random() <= eta * xi * dt:
            for mi in range(m):
                j = random.choice(range(N))
                while j == i or G.has_edge(i, j):
                    j = random.choice(range(N))
                G.add_edge(i, j)
    return G  # return a graph object

# Generating T time steps time-varying activity driven networks
def temporal_activityDriven(N=1000, m=6, T=5):
    """"
    Parameters
    ----------
    N :   nodes
    m :   rewiring links
    T :   time slots
    """
    tG = {}
    for i in range(1, T+1):
        G_temp = init_activityDriven(N, m)
        tempNet = nx.to_dict_of_dicts(G_temp)
        tG[i] = tempNet
    return tG  # return a dict-like time-varying graph


def temporal_activityDriven_withMemory(N=10000, m=4, T=5, r=0):
    """"
    Parameters
    ----------
    N :   nodes
    m :   rewiring links
    T :   time slots
    r :   memory rate, keeping previous links
    """
    # initiate parameters
    # N = 100   # nodes
    # T = 1000  # time slots
    # r = 0.1   # memory rate, keeping previous links
    dt = 1    # active slot
    eta = 10  # rescaling factor , ai = eta * xi

    G0 = emptyGraph(N)  # initial graph
    G1 = G0   # current timeslot graph
    tG = {}  # graph dicts
    for ti in range(1, T+1):
        G2 = G1  # previous timeslot graph
        G1 = emptyGraph(N)  # current timeslot graph
        for i in range(N):
            xi = 0.1  # node's activated prob belongs to F(x)=x**(-3)
#             print xi*eta
            if random.random() < eta * xi * dt:  # node i is active
#                 ki = nx.degree(G2, i)            # calculate degree of node i
                neighbors = G2.neighbors(i)      # neighbors list of node i
                # link selection of each neighbor
                if len(neighbors) > 0:
                    for neighbor in neighbors:
                        if random.random() < r:
                            # randomly keep one previous neighbor
                            j = neighbor
                            G1.add_edge(i, j)
                        else:
                            # randomly select one other node
                            # G1.remove_edge(i,neighbor)
                            j = random.choice(list(set(range(N)) - {i} - set(neighbors)))
                            count = 0
                            while G1.has_edge(i, j):
                                j = random.choice(list(set(range(N)) - {i} - set(neighbors)))
                                count += 1
                            G1.add_edge(i, j)
                else:
                    for mi in range(m):
                        j = random.choice(range(N))
                        while j == i or G1.has_edge(i, j):
                            j = random.choice(range(N))
                        G1.add_edge(i, j)
                    
        tempNet = nx.to_dict_of_dicts(G1)
        tG[ti] = tempNet
    return tG  # return a dict-like time-varying graph

# main process to constructing time varynetworks
if __name__ == "__main__":

    # tG = temporal_activityDriven_withMemory(1000, 6, 10, 0.1)
    #tG = temporal_activityDriven(1000, 6, 10)

    # initiate network parameters
    N = 10000
    m = 4
    T = 1000
    #p = 0.0

    for i in np.arange(0, 1.1, 0.1):

        networkPath = "F:\\ActivityDrivenNetwork\\ADM_N_%s_m_%s_T_%s_p_%s_sameA.json"\
                                        %(N, m, T, i)
    
        tNet = temporal_activityDriven_withMemory(N, m, T, i)
        with open(networkPath, 'w') as netFile:
            netFile.write(json.dumps(tNet))
        print "network %s has been constructed!" %i

    # ------------counting running time-----------------
    endTime = time.clock()
    print "Running time: %s Seconds(%s Minutes, %s Hours)" % (
        endTime - beginTime, (endTime - beginTime) / 60, (endTime - beginTime) / 3600)
        